<a href="https://colab.research.google.com/github/daishwary/ImageClassifier/blob/master/sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



A model to predict the value of the question in the TV game show  “Jeopardy!”.


In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalMaxPooling1D, LSTM, Bidirectional, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Preprocessing

The first step is to load the data (in CSV format), and split the data.

In [2]:
data_df = pd.read_csv('/content/JEOPARDY_CSV.csv')
data_df = data_df[data_df[' Value'] != 'None']

print(data_df.shape)
data_df.head()

(213296, 7)


,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


## Creating bins

Since the values could easily vary, this means we would have way too many classes to classify! Instead, we will bin it in this way: if the value is smaller than 1000, then we round to the nearest hundred. Otherwise, if it's between 1000 and 10k, we round it to nearest thousand. If it's greater than 10k, then we round it to the nearest 10-thousand.

In [3]:
data_df['ValueNum'] = data_df[' Value'].apply(
    lambda value: int(value.replace(',', '').replace('$', ''))
)

In [4]:
def binning(value):
    if value < 1000:
        return np.round(value, -2)
    elif value < 10000:
        return np.round(value, -3)
    else:
        return np.round(value, -4)

data_df['ValueBins'] = data_df['ValueNum'].apply(binning)

In [5]:
print("Total number of categories:", data_df[' Value'].unique().shape[0])
print("Number of categories after binning:", data_df['ValueBins'].unique().shape[0])
print("\nBinned Categories:", data_df['ValueBins'].unique())

Total number of categories: 149
Number of categories after binning: 21

Binned Categories: [  200   400   600   800  2000  1000  3000  5000   100   300   500  4000
  7000   700  8000  6000 10000   900  9000     0 20000]


Then, we will split our data by randomly selected 20% of the shows, and use the questions from that show as what we will try to predict.

In [6]:
show_numbers = data_df['Show Number'].unique()
train_shows, test_shows = train_test_split(show_numbers, test_size=0.2, random_state=2019)

train_mask = data_df['Show Number'].isin(train_shows)
test_mask = data_df['Show Number'].isin(test_shows)

train_labels = data_df.loc[train_mask, 'ValueBins']
train_questions = data_df.loc[train_mask, ' Question']
test_labels = data_df.loc[test_mask, 'ValueBins']
test_questions = data_df.loc[test_mask, ' Question']

# 2. Simple Linear Model

## Transform questions to bag-of-words

Bag of words is a very simple, but very convenient way of representing any type of freeform text using vectors.

In our model, we will limit ourselves to only using the top 2000 most frequent words as features, in order for the logistic regression model to not overfit on too many features. Further, we are removing **stop words**, which are very common words in English that we wish to remove in order to only keep relevant information.


In [7]:
%%time
bow = CountVectorizer(stop_words='english', max_features=2000)
bow.fit(data_df[' Question'])

CPU times: user 3.15 s, sys: 29.7 ms, total: 3.18 s
Wall time: 3.19 s


In [8]:
X_train = bow.transform(train_questions)
X_test = bow.transform(test_questions)

y_train = train_labels
y_test = test_labels

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (170704, 2000)
Shape of X_test: (42592, 2000)
Shape of y_train: (170704,)
Shape of y_test: (42592,)


## Train the Logistic Regression model

Logistic Regression is perhaps the simplest regression model out there.

In [9]:
%%time
lr = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=200)
lr.fit(X_train, y_train)

CPU times: user 1min 10s, sys: 22.5 ms, total: 1min 10s
Wall time: 1min 10s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


## Evaluate the results

In [10]:
y_pred = lr.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
         100       0.05      0.00      0.01      1863
         200       0.18      0.14      0.15      6132
         300       0.06      0.00      0.01      1801
         400       0.21      0.57      0.30      8425
         500       0.10      0.01      0.02      1827
         600       0.11      0.01      0.02      4099
         700       0.00      0.00      0.00        41
         800       0.15      0.10      0.12      6279
         900       0.00      0.00      0.00        28
        1000       0.19      0.20      0.20      6720
        2000       0.19      0.10      0.13      4938
        3000       0.00      0.00      0.00       198
        4000       0.00      0.00      0.00       121
        5000       0.00      0.00      0.00        61
        6000       0.00      0.00      0.00        21
        7000       0.00      0.00      0.00         9
        8000       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 2. LSTM

## Tokenize & Pad

We are doing 3 things here:

1. Train a tokenizer in all the text. This tokenizer will create an dictionary mapping words to an index, aka `tokenizer.word_index`.
2. Convert the questions (which are strings of text) into a list of list of integers, each representing the index of a word in the `word_index`.
3. Pad each "list of list" into a single numpy array. To do this, we use the `pad_sequences` function, and set a maximum length (50 is reasonable since most questions will be at most 20 words), after which any word is cutoff.

Note:
* Tokenizer will take at most 50k words. Here, we are using more words than Logistic Regression since the input dimension does not account for **all** words, but only the words that are actually given in the sequence.

In [11]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(data_df[' Question'])

train_sequence = tokenizer.texts_to_sequences(train_questions)
test_sequence = tokenizer.texts_to_sequences(test_questions)

print("Original text:", train_questions[0])
print("Converted sequence:", train_sequence[0])

Original text: For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory
Converted sequence: [7, 1, 112, 272, 102, 4, 14, 189, 7842, 9, 226, 173, 5422, 7, 41554, 2, 571, 1552]


In [12]:
X_train = pad_sequences(train_sequence, maxlen=50)
X_test = pad_sequences(test_sequence, maxlen=50)

print(X_train.shape)
print(X_test.shape)

(170704, 50)
(42592, 50)


## Encode labels as counts


In [13]:
le = LabelEncoder()
le.fit(data_df['ValueBins'])

y_train = le.transform(train_labels)
y_test = le.transform(test_labels)

print(y_train.shape)
print(y_test.shape)

(170704,)
(42592,)


## Building and running the model

In [14]:
num_words = tokenizer.num_words
output_size = len(le.classes_)

In [15]:
model = Sequential([
    Embedding(input_dim=num_words, 
              output_dim=200, 
              mask_zero=True, 
              input_length=50),
    Bidirectional(LSTM(150, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(300, activation='relu'),
    Dropout(0.5),
    Dense(output_size, activation='softmax')
    
])

model.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 200)           10000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 300)           421200    
_________________________________________________________________
global_max_pooling1d (Global (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               90300     
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 21)                6321      
Total params: 10,517,821
Trainable params: 10,517,821
Non-trainable params: 0
____________________________________________

## Train the model

In [16]:
model.fit(X_train, y_train, epochs=10, batch_size=1024, validation_split=0.1)

Epoch 1/10
151/151 [==============================] - 542s 4s/step - loss: 2.4098 - accuracy: 0.1676 - val_loss: 2.1073 - val_accuracy: 0.2068
Epoch 2/10
151/151 [==============================] - 525s 3s/step - loss: 2.0927 - accuracy: 0.2091 - val_loss: 2.0927 - val_accuracy: 0.2161
Epoch 3/10
151/151 [==============================] - 542s 4s/step - loss: 1.9770 - accuracy: 0.2531 - val_loss: 2.1355 - val_accuracy: 0.1972
Epoch 4/10
151/151 [==============================] - 537s 4s/step - loss: 1.7786 - accuracy: 0.3455 - val_loss: 2.2707 - val_accuracy: 0.1824
Epoch 5/10
151/151 [==============================] - 541s 4s/step - loss: 1.5133 - accuracy: 0.4640 - val_loss: 2.5372 - val_accuracy: 0.1799
Epoch 6/10
151/151 [==============================] - 542s 4s/step - loss: 1.2424 - accuracy: 0.5670 - val_loss: 3.0529 - val_accuracy: 0.1726
Epoch 7/10
151/151 [==============================] - 550s 4s/step - loss: 1.0186 - accuracy: 0.6466 - val_loss: 3.5554 - val_accuracy: 0.1727

## Evaluate the model

In [17]:
y_pred = model.predict(X_test, batch_size=1024).argmax(axis=1)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.07      0.05      0.06      1863
           2       0.18      0.20      0.19      6132
           3       0.06      0.04      0.05      1801
           4       0.21      0.19      0.20      8425
           5       0.07      0.06      0.06      1827
           6       0.11      0.10      0.10      4099
           7       0.00      0.00      0.00        41
           8       0.16      0.17      0.16      6279
           9       0.00      0.00      0.00        28
          10       0.19      0.21      0.20      6720
          11       0.17      0.18      0.17      4938
          12       0.01      0.01      0.01       198
          13       0.00      0.00      0.00       121
          14       0.00      0.00      0.00        61
          15       0.00      0.00      0.00        21
          16       0.00      0.00      0.00         9
          17       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
